In [2]:
#!pip install pandas
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

In [3]:
xmlfiles = glob('./data_images/*.xml')


In [14]:
#Data cleaning
xmlfiles = list(map(lambda x: x.replace('\\','/'),xmlfiles))
xmlfiles

['./data_images/000001.xml',
 './data_images/000002.xml',
 './data_images/000007.xml',
 './data_images/000009.xml',
 './data_images/000012.xml',
 './data_images/000016.xml',
 './data_images/000017.xml',
 './data_images/000019.xml',
 './data_images/000020.xml',
 './data_images/000021.xml',
 './data_images/000023.xml',
 './data_images/000024.xml',
 './data_images/000026.xml',
 './data_images/000030.xml',
 './data_images/000032.xml',
 './data_images/000033.xml',
 './data_images/000034.xml',
 './data_images/000035.xml',
 './data_images/000036.xml',
 './data_images/000039.xml',
 './data_images/000041.xml',
 './data_images/000042.xml',
 './data_images/000044.xml',
 './data_images/000046.xml',
 './data_images/000047.xml',
 './data_images/000048.xml',
 './data_images/000050.xml',
 './data_images/000051.xml',
 './data_images/000052.xml',
 './data_images/000060.xml',
 './data_images/000061.xml',
 './data_images/000063.xml',
 './data_images/000064.xml',
 './data_images/000065.xml',
 './data_image

In [31]:
def extract_text(filename):
    tree = et.parse(filename)
    
    root = tree.getroot()
    image_name = root.find('filename').text
    height = root.find('size').find('height').text
    width = root.find('size').find('width').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name,width,height,name,xmin,xmax,ymin,ymax])
    
    return parser

In [44]:
parser_all = list(map(extract_text,xmlfiles))
data = reduce(lambda x,y: x+y,parser_all)

df = pd.DataFrame(data,columns = ['filename','width','height','name','xmin','xmax','ymin','ymax'])
df.head()

filename width height name xmin  xmax ymin ymax
0  000001.jpg  1024    657  car   14   301  335  522
1  000001.jpg  1024    657  car  269   571  345  489
2  000001.jpg  1024    657  car  502   798  342  450
3  000001.jpg  1024    657  car  709  1009  333  438
4  000002.jpg   800    600  car   41   768  240  497

In [54]:
cols = ['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(int)

df['center_x'] = (df['xmax']+df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax']+df['ymin'])/(2*df['height'])

df['w'] = (df['xmax']-df['xmin'])/df['width']
df['h'] = (df['ymax']-df['ymin'])/df['height']

df.head()

filename  width  height name  xmin  xmax  ymin  ymax  center_x  center_y  \
0  000001.jpg   1024     657  car    14   301   335   522  0.153809  0.652207   
1  000001.jpg   1024     657  car   269   571   345   489  0.410156  0.634703   
2  000001.jpg   1024     657  car   502   798   342   450  0.634766  0.602740   
3  000001.jpg   1024     657  car   709  1009   333   438  0.838867  0.586758   
4  000002.jpg    800     600  car    41   768   240   497  0.505625  0.614167   

          w         h  
0  0.280273  0.284627  
1  0.294922  0.219178  
2  0.289062  0.164384  
3  0.292969  0.159817  
4  0.908750  0.428333

In [69]:
images = df['filename'].unique()
img_df = pd.DataFrame(images,columns=['filename'])
img_df.head()

img_train = tuple(img_df.sample(frac=0.8)['filename'])

img_test = tuple(img_df.query(f'filename not in {img_train}')['filename'])
len(img_train),len(img_test)

(4010, 1002)

In [73]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

4.150608352515619

In [75]:
def label_encoding(x):
    labels = {
        'person':0,'car':1,'chair':2,'bottle':3,'pottedplant':4,'bird':5,
        'dog':6,'sofa':7,'bicycle':8,'horse':9,'boat':10,'motorbike':11,
        'cat':12,'tvmonitor':13, 'cow':14,'sheep':15,'aeroplane':16,
        'train':17,'diningtable':18,'bus':19
    }
    return labels[x]

In [80]:
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

train_df.head(10)

C:\Users\James\AppData\Local\Temp\ipykernel_30904\691076782.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id'] = train_df['name'].apply(label_encoding)
C:\Users\James\AppData\Local\Temp\ipykernel_30904\691076782.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id'] = test_df['name'].apply(label_encoding)


filename  width  height    name  xmin  xmax  ymin  ymax  center_x  \
0   000001.jpg   1024     657     car    14   301   335   522  0.153809   
1   000001.jpg   1024     657     car   269   571   345   489  0.410156   
2   000001.jpg   1024     657     car   502   798   342   450  0.634766   
3   000001.jpg   1024     657     car   709  1009   333   438  0.838867   
4   000002.jpg    800     600     car    41   768   240   497  0.505625   
5   000002.jpg    800     600     car   533   722   236   299  0.784375   
6   000007.jpg    500     333     car   141   500    50   330  0.641000   
13  000017.jpg    480     364  person   185   279    62   199  0.483333   
14  000017.jpg    480     364   horse    90   403    78   336  0.513542   
17  000020.jpg    375     500     car    33   371   148   416  0.538667   

    center_y         w         h  id  
0   0.652207  0.280273  0.284627   1  
1   0.634703  0.294922  0.219178   1  
2   0.602740  0.289062  0.164384   1  
3   0.586758  0.292969  0.159817   1  
4   0.614167  0.908750  0.428333   1  
5   0.445833  0.236250  0.105000   1  
6   0.570571  0.718000  0.840841   1  
13  0.358516  0.195833  0.376374   0  
14  0.568681  0.652083  0.708791   9  
17  0.564000  0.901333  0.536000   1

In [85]:
import os
from shutil import move

In [88]:
train_folder = 'data_images/train'
test_folder = 'data_images/test'

#os.mkdir(train_folder)
#os.mkdir(test_folder)

cols = ['filename','id','center_x','center_y','w','h']

groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [103]:
def save_data(filename,folder_path,group_obj):
    src = os.path.join('data_images',filename)
    dst = os.path.join(folder_path,filename)
    move(src,dst)
    
    text_filename = os.path.join(folder_path,
                                 os.path.splitext(filename)[0]+ '.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep = ' ', index = False, header = False)

In [104]:
filename_series = pd.Series(groupby_obj_train.groups.keys())

In [107]:

filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

0       None
1       None
2       None
3       None
4       None
        ... 
4005    None
4006    None
4007    None
4008    None
4009    None
Length: 4010, dtype: object

In [110]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data,args=(test_folder,groupby_obj_test))

0       None
1       None
2       None
3       None
4       None
        ... 
997     None
998     None
999     None
1000    None
1001    None
Length: 1002, dtype: object